In [10]:
from pymongo.mongo_client import MongoClient

url = "mongodb://localhost:27017"

client = MongoClient(url)

In [5]:
client.list_database_names()

['admin', 'config', 'local', 'mydb']

In [8]:
database = client["youtube_test"]
database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'youtube_test')

In [11]:
client.list_database_names()

['admin', 'config', 'local', 'mydb']

In [12]:
database.list_collection_names()

[]

In [13]:
collection = database["topics"]
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'youtube_test'), 'topics')

In [14]:
database.list_collection_names()

[]

In [15]:
data = {"Video": "Test1", "author": "Diljyot Singh"}
collection.insert_one(data)
client.list_database_names()

['admin', 'config', 'local', 'mydb', 'youtube_test']

In [16]:
client.drop_database("mydb")

In [17]:
client.list_database_names()

['admin', 'config', 'local', 'youtube_test']

In [19]:
all_records = collection.find()

In [20]:
for id, record in enumerate(all_records):
    print(id, record)

0 {'_id': ObjectId('65e104ca79ce90dc74b67fd3'), 'Video': 'Test1', 'author': 'Diljyot Singh'}


In [21]:
data = [{"Video": "Test2", "author": "Diljyot Singh"},
        {"Video": "Test3", "description": "Its a test Video"}]
collection.insert_many(data)
all_records = collection.find()
for id, record in enumerate(all_records):
    print(id, record)

0 {'_id': ObjectId('65e104ca79ce90dc74b67fd3'), 'Video': 'Test1', 'author': 'Diljyot Singh'}
1 {'_id': ObjectId('65e106bf79ce90dc74b67fd4'), 'Video': 'Test2', 'author': 'Diljyot Singh'}
2 {'_id': ObjectId('65e106bf79ce90dc74b67fd5'), 'Video': 'Test3', 'description': 'Its a test Video'}


In [ ]:
from typing import Any
import os
import pandas as pd
from pymongo.mongo_client import MongoClient
import json
from ensure import ensure_annotations


class mongo_operation:
    __collection=None # here i have created a private/protected variable
    __database=None
    
    def __init__(self,client_url: str, database_name: str, collection_name: str=None):
        self.client_url=client_url
        self.database_name=database_name
        self.collection_name=collection_name
       
    def create_mongo_client(self,collection=None):
        client=MongoClient(self.client_url)
        return client
    
    def create_database(self,collection=None):
        if mongo_operation.__database==None:
            client=self.create_mongo_client(collection)
            self.database=client[self.database_name]
        return self.database 
    
    def create_collection(self,collection=None):
        if mongo_operation.__collection==None:
            database=self.create_database(collection)
            self.collection=database[self.collection_name]
            mongo_operation.__collection=collection
        
        if mongo_operation.__collection!=collection:
            database=self.create_database(collection)
            self.collection=database[self.collection_name]
            mongo_operation.__collection=collection
            
        return self.collection
    
    def insert_record(self,record: dict, collection_name: str) -> Any:
        if type(record) == list:
            for data in record:
                if type(data) != dict:
                    raise TypeError("record must be in the dict")    
            collection=self.create_collection(collection_name)
            collection.insert_many(record)
        elif type(record)==dict:
            collection=self.create_collection(collection_name)
            collection.insert_one(record)
    
    def bulk_insert(self,datafile,collection_name:str=None):
        self.path=datafile
        
        if self.path.endswith('.csv'):
            pd.read_csv(self.path,encoding='utf-8')
            
        elif self.path.endswith(".xlsx"):
            dataframe=pd.read_excel(self.path,encoding='utf-8')
            
        datajson=json.loads(dataframe.to_json(orient='record'))
        collection=self.create_collection(collection_name)
        collection.insert_many(datajson)